# <center>IME639A Project Assignment (Bus Driver Scheduling Problem)

In [1]:
from collections import defaultdict
from pprint import pprint
import random
from pulp import *

## The Problem
So we are aiming to model and solve the following problem.

There are five bus routes, namely A, B, C, D and E; each of which is a closed loop in itself. Each route takes an hour to complete. A single bus runs on each route starting from 5AM in the morning until 11PM in the evening. Moreover, all the routes start and end at a common point every hour on the hour, making it convenient for the drivers to take their meal breaks and shift changes at that location. Since our bus service runs from 5 AM until 11 PM, and each route takes an hour to complete, we’ll have 18 full trips for each route (90 total trips).

### Generating all trips
The following subroutine generates and lists all the the 90 trips in our route.

In [2]:
def list_all_trips(route_names, start_hour, end_hour):
  results = []
  for route_name in route_names:
    for hour in range(start_hour, end_hour):
      results.append((route_name, hour))
  return results

trips=list_all_trips('ABCDE', 5, 18 + 5)
print(trips)

[('A', 5), ('A', 6), ('A', 7), ('A', 8), ('A', 9), ('A', 10), ('A', 11), ('A', 12), ('A', 13), ('A', 14), ('A', 15), ('A', 16), ('A', 17), ('A', 18), ('A', 19), ('A', 20), ('A', 21), ('A', 22), ('B', 5), ('B', 6), ('B', 7), ('B', 8), ('B', 9), ('B', 10), ('B', 11), ('B', 12), ('B', 13), ('B', 14), ('B', 15), ('B', 16), ('B', 17), ('B', 18), ('B', 19), ('B', 20), ('B', 21), ('B', 22), ('C', 5), ('C', 6), ('C', 7), ('C', 8), ('C', 9), ('C', 10), ('C', 11), ('C', 12), ('C', 13), ('C', 14), ('C', 15), ('C', 16), ('C', 17), ('C', 18), ('C', 19), ('C', 20), ('C', 21), ('C', 22), ('D', 5), ('D', 6), ('D', 7), ('D', 8), ('D', 9), ('D', 10), ('D', 11), ('D', 12), ('D', 13), ('D', 14), ('D', 15), ('D', 16), ('D', 17), ('D', 18), ('D', 19), ('D', 20), ('D', 21), ('D', 22), ('E', 5), ('E', 6), ('E', 7), ('E', 8), ('E', 9), ('E', 10), ('E', 11), ('E', 12), ('E', 13), ('E', 14), ('E', 15), ('E', 16), ('E', 17), ('E', 18), ('E', 19), ('E', 20), ('E', 21), ('E', 22)]


Our goal is to create a roster of duties for the bus drivers, where each duty is a sequence of trips that a driver is required to operate. It is essential to ensure that **every trip is included in precisely one driver's duty**.

We have some guidelines from payroll that we need to consider while creating the duties for the bus drivers:

1. **Each duty over four hours should allow the driver to have a break**.
2. **The drivers will receive payment for each hour they work, excluding their break time**.
3. **The drivers will receive guaranteed pay for eight hours of work, irrespective of the actual number of hours they work**.
4. **If a driver works for more than eight hours, they will receive overtime pay, which is 1.5 times their regular pay rate**.

### Optimization Task
Our task is to **create a roster of duties which minimizes the driving costs**.

### Generating the duties
Generating all possible duties is an exponentially complex problem. For instance in case of our problem we have 90 trips, the possible number of duties is = 2^(90). This is enormously large!!

Therefore, we will randomly create only a certain number duties from the set of trips. We have already written code above to generate trips above, and we will now write a function to randomly generate duties while ensuring that **no duty includes multiple trips that occur simultaneously**.

In [3]:
def generate_duties(num_duties, trips):
  possible_hours = {t[1] for t in trips}
  possible_routes_for_hour = defaultdict(list)
  for t in trips:
    possible_routes_for_hour[t[1]].append(t[0])

  duties = []

  for _ in range(num_duties):
    num_trips = random.randrange(1, len(possible_hours) + 1)
    hours = random.sample(possible_hours, num_trips)
    duty = []
    for hour in sorted(hours):
      route_name = random.choice(possible_routes_for_hour[hour])
      duty.append((route_name, hour))
    duties.append(duty)

  return duties

duties_5 = generate_duties(5, trips) #generating 5 duties - this is just for testing if code works...
pprint(duties_5)
duties_100 = generate_duties(100, trips) #generating 100 duties
duties_1000 = generate_duties(1000, trips) #generating 1000 duties

[[('E', 14)],
 [('C', 5),
  ('E', 6),
  ('B', 7),
  ('C', 8),
  ('A', 9),
  ('B', 10),
  ('B', 11),
  ('A', 12),
  ('E', 13),
  ('E', 14),
  ('D', 16),
  ('C', 17),
  ('B', 18),
  ('B', 19),
  ('C', 20),
  ('A', 21),
  ('A', 22)],
 [('C', 7), ('A', 15)],
 [('A', 16)],
 [('E', 5), ('E', 6), ('A', 11), ('B', 12), ('D', 14), ('A', 19), ('A', 22)]]


/tmp/ipykernel_33697/1994482546.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  hours = random.sample(possible_hours, num_trips)


### Assigning the cost to each duty we generated above.

It is to be noted that while generating the above duties we did not take into account the any of the payroll constraints. 

**The first constraint of payroll puts a limit of 4 on the consecutive number of hours a driver can work.** Each duty over 4 hours should be followed by a rest. So those duties generated above, which violate this contraint cannot be considered as valid ones. Since our problem is an Optimization-Minimization problem, we have an easy way out of this problem is to assign a very very high cost to such duties so that they are not counted into the optimal solution. 

Other three constraints just determine the pay for work, do not impose any restriction on the nature of duties. They have been used to determine the cost in the code below.

In [4]:
def cost(duty):
  #assigning 1 unit as a standard pay for 1 hour of work.

  # Drivers get paid for each hour they work, with minimum pay equalling to that of 8 hours a day.
  pay = max(len(duty), 8)
  # for working more than 8 hours a day they get 1.5 times the pay of each hour
  if len(duty) > 8:
    pay += 0.5 * (len(duty) - 8)

  # If there's no break, set pay to a very large value, since it isn't a feasible duty. 
  #This will drop the duty from incoming into the optimal solution.
  if len(duty) > 4:
    first_hour = duty[0][1]
    last_hour = duty[-1][1]
    if last_hour - first_hour + 1 == len(duty):
      pay = 1e7

  return pay

#cost for 5 duties - this is just for testing if code works properly...
duties_with_costs_5 = [(cost(duty), duty) for duty in duties_5]
pprint(duties_with_costs_5)

duties_with_costs_100 = [(cost(duty), duty) for duty in duties_100]
duties_with_costs_1000 = [(cost(duty), duty) for duty in duties_1000]

[(8, [('E', 14)]),
 (21.5,
  [('C', 5),
   ('E', 6),
   ('B', 7),
   ('C', 8),
   ('A', 9),
   ('B', 10),
   ('B', 11),
   ('A', 12),
   ('E', 13),
   ('E', 14),
   ('D', 16),
   ('C', 17),
   ('B', 18),
   ('B', 19),
   ('C', 20),
   ('A', 21),
   ('A', 22)]),
 (8, [('C', 7), ('A', 15)]),
 (8, [('A', 16)]),
 (8,
  [('E', 5), ('E', 6), ('A', 11), ('B', 12), ('D', 14), ('A', 19), ('A', 22)])]


### Finally solving the problem

Before moving on to solve the problem we need to ensure two more things:

1. **Our objective is to create a set of duties that includes all the trips.** However, as we are randomly generating duties, we cannot guarantee this. There is a possibility that all randomly generated duties may not include at all trip on a particular route. To address this issue, we can generate one duty for each trip, which only includes that specific trip. This ensures that each trip is included in at least once in the set of duties.

2. In order to ensure that each trip is only assigned to one duty, we can impose a constraint on each trip such that the corresponding variables representing the duties that the trip can be assigned to, sum up to exactly 1. This constraint ensures that **every trip is assigned to one and only one duty, and not assigned to multiple duties or left unassigned**.

Now after generating the set of possible duties and their associated costs, we move on on solve the problem. That is to select those duties that minimize the cost, simultaneously satisying all the contraints of the problem.

In [5]:
def solve(duties, trips):
  problem = LpProblem('bus_driver_scheduling', LpMinimize)
  variables = []
  costs = []
  variables_for_trip = {trip: [] for trip in trips} # To generate and store all the constraints for each trip.

  #to meet the stated requirement in 1. of the write-up just above this code block
  duties = duties + [[trip] for trip in trips]

  # Gather up variables and costs
  for i, duty in enumerate(duties):
    # Create a binary variable (the value can be only 0 or 1)
    x = LpVariable('x{}'.format(i + 1), 0, 1, LpBinary)
    variables.append(x)
    costs.append(cost(duty))
    for trip in duty:
      variables_for_trip[trip].append(x)

  # Create the objective function. lpDot is shorthand for cost*x for (cost, x) in zip(costs, variables)
  problem += lpDot(costs, variables)

  #to meet the stated requirement in 2. of the write-up just above this code block
  for xs in variables_for_trip.values():
    problem += lpSum(xs) == 1

  status = problem.solve()

  # We now have a solution. Those duties with values of x's=1 are to be uses. Sum the cost for each used duty.
  solution = []
  total_cost = 0
  for i, x in enumerate(variables):
    if x.value():
      solution.append(duties[i])
      total_cost += costs[i]

  return solution, total_cost

In [6]:
solution_duties, solution_cost = solve(duties_5, trips)
print("Cost: {}".format(solution_cost))
pprint(solution_duties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jai_bhawani/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3a2cf9f13d904b2595420778f4f21bf3-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/3a2cf9f13d904b2595420778f4f21bf3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 95 COLUMNS
At line 499 RHS
At line 590 BOUNDS
At line 686 ENDATA
Problem MODEL has 90 rows, 95 columns and 118 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 597.5 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 2 substitutions
Cgl0004I processed model has 1 rows, 3 columns (3 integer (3 of which binary)) and 3 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 597.5
Cbc0038I Before mini branch and bound, 3 

In [7]:
solution_duties, solution_cost = solve(duties_100, trips)
print("Cost: {}".format(solution_cost))
pprint(solution_duties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jai_bhawani/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/976e7dce56084296990f27d41ef4eb93-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/976e7dce56084296990f27d41ef4eb93-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 95 COLUMNS
At line 1721 RHS
At line 1812 BOUNDS
At line 2003 ENDATA
Problem MODEL has 90 rows, 190 columns and 1055 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 187.309 - 0.00 seconds
Cgl0004I processed model has 90 rows, 185 columns (185 integer (185 of which binary)) and 1050 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 90 integers unsatisfied sum - 16.9508
Cbc0038I Pass   1: suminf.   15.37974 (90) obj. 5.29783e+06 iterations 32
Cbc0038I Pass   2: suminf.   14.83328 (89) obj. 7.5

In [8]:
solution_duties, solution_cost = solve(duties_1000, trips)
print("Cost: {}".format(solution_cost))
pprint(solution_duties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jai_bhawani/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/afe25dacd41d4694a23a2138a8fe84a4-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/afe25dacd41d4694a23a2138a8fe84a4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 95 COLUMNS
At line 13200 RHS
At line 13291 BOUNDS
At line 14382 ENDATA
Problem MODEL has 90 rows, 1090 columns and 9834 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 94.262 - 0.02 seconds
Cgl0004I processed model has 90 rows, 1037 columns (1037 integer (1037 of which binary)) and 9780 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 90 integers unsatisfied sum - 10.5687
Cbc0038I Pass   1: suminf.    5.22289 (90) obj. 2.62451e+07 iterations 280
Cbc0038I Pass   2: suminf.    5.67461 (90) o

Cbc0010I After 30000 nodes, 8234 on tree, 216 best solution, best possible 110.83393 (131.87 seconds)
Cbc0010I After 31000 nodes, 8710 on tree, 216 best solution, best possible 111.26646 (137.43 seconds)
Cbc0010I After 32000 nodes, 9188 on tree, 216 best solution, best possible 111.65083 (142.88 seconds)
Cbc0038I Full problem 90 rows 1037 columns, reduced to 72 rows 89 columns
Cbc0010I After 33000 nodes, 9669 on tree, 216 best solution, best possible 112.09398 (148.36 seconds)
Cbc0010I After 34000 nodes, 9674 on tree, 216 best solution, best possible 112.09398 (149.21 seconds)
Cbc0010I After 35000 nodes, 10156 on tree, 216 best solution, best possible 112.43538 (154.72 seconds)
Cbc0038I Full problem 90 rows 1037 columns, reduced to 74 rows 104 columns
Cbc0010I After 36000 nodes, 10650 on tree, 216 best solution, best possible 112.43538 (159.60 seconds)
Cbc0004I Integer solution of 209 found after 3648836 iterations and 36173 nodes (159.79 seconds)
Cbc0010I After 37000 nodes, 10985 on t

Cbc0010I After 101000 nodes, 22362 on tree, 195 best solution, best possible 112.44028 (349.80 seconds)
Cbc0010I After 102000 nodes, 22382 on tree, 195 best solution, best possible 112.44028 (352.19 seconds)
Cbc0010I After 103000 nodes, 22374 on tree, 195 best solution, best possible 112.44028 (354.59 seconds)
Cbc0010I After 104000 nodes, 22360 on tree, 195 best solution, best possible 112.44028 (357.11 seconds)
Cbc0010I After 105000 nodes, 22360 on tree, 195 best solution, best possible 112.44028 (360.50 seconds)
Cbc0010I After 106000 nodes, 22348 on tree, 195 best solution, best possible 112.44028 (363.93 seconds)
Cbc0010I After 107000 nodes, 22356 on tree, 195 best solution, best possible 112.44028 (367.08 seconds)
Cbc0010I After 108000 nodes, 22350 on tree, 195 best solution, best possible 112.44028 (370.12 seconds)
Cbc0010I After 109000 nodes, 22366 on tree, 195 best solution, best possible 112.44028 (373.56 seconds)
Cbc0010I After 110000 nodes, 22364 on tree, 195 best solution, b

Cbc0010I After 178000 nodes, 22344 on tree, 195 best solution, best possible 112.44028 (565.92 seconds)
Cbc0010I After 179000 nodes, 22318 on tree, 195 best solution, best possible 112.44028 (568.50 seconds)
Cbc0010I After 180000 nodes, 22367 on tree, 195 best solution, best possible 112.44028 (571.80 seconds)
Cbc0010I After 181000 nodes, 22349 on tree, 195 best solution, best possible 112.44028 (574.94 seconds)
Cbc0010I After 182000 nodes, 22357 on tree, 195 best solution, best possible 112.44028 (577.94 seconds)
Cbc0010I After 183000 nodes, 22349 on tree, 195 best solution, best possible 112.44028 (580.60 seconds)
Cbc0010I After 184000 nodes, 22337 on tree, 195 best solution, best possible 112.44028 (583.14 seconds)
Cbc0010I After 185000 nodes, 22359 on tree, 195 best solution, best possible 112.44028 (585.87 seconds)
Cbc0010I After 186000 nodes, 22345 on tree, 195 best solution, best possible 112.44028 (588.53 seconds)
Cbc0010I After 187000 nodes, 22355 on tree, 195 best solution, b

Cbc0010I After 255000 nodes, 22337 on tree, 195 best solution, best possible 112.44028 (769.62 seconds)
Cbc0010I After 256000 nodes, 22331 on tree, 195 best solution, best possible 112.44028 (772.20 seconds)
Cbc0010I After 257000 nodes, 22329 on tree, 195 best solution, best possible 112.44028 (774.72 seconds)
Cbc0010I After 258000 nodes, 22339 on tree, 195 best solution, best possible 112.44028 (777.31 seconds)
Cbc0010I After 259000 nodes, 22333 on tree, 195 best solution, best possible 112.44028 (780.10 seconds)
Cbc0010I After 260000 nodes, 22322 on tree, 195 best solution, best possible 112.44028 (782.69 seconds)
Cbc0010I After 261000 nodes, 22334 on tree, 195 best solution, best possible 112.44028 (785.46 seconds)
Cbc0010I After 262000 nodes, 22332 on tree, 195 best solution, best possible 112.44028 (787.84 seconds)
Cbc0010I After 263000 nodes, 22330 on tree, 195 best solution, best possible 112.44028 (790.21 seconds)
Cbc0010I After 264000 nodes, 22344 on tree, 195 best solution, b

...

Cbc0010I After 30828000 nodes, 22026 on tree, 191.5 best solution, best possible 113.11938 (70321.20 seconds)
Cbc0010I After 30829000 nodes, 22038 on tree, 191.5 best solution, best possible 113.11938 (70323.13 seconds)
Cbc0010I After 30830000 nodes, 22006 on tree, 191.5 best solution, best possible 113.11938 (70325.09 seconds)
Cbc0010I After 30831000 nodes, 22006 on tree, 191.5 best solution, best possible 113.11938 (70327.04 seconds)
Cbc0010I After 30832000 nodes, 22000 on tree, 191.5 best solution, best possible 113.11938 (70329.00 seconds)
Cbc0010I After 30833000 nodes, 22007 on tree, 191.5 best solution, best possible 113.11938 (70330.87 seconds)
Cbc0010I After 30834000 nodes, 22000 on tree, 191.5 best solution, best possible 113.11938 (70332.76 seconds)
Cbc0010I After 30835000 nodes, 22024 on tree, 191.5 best solution, best possible 113.11938 (70334.78 seconds)
Cbc0010I After 30836000 nodes, 22028 on tree, 191.5 best solution, best possible 113.11938 (70336.94 seconds)
Cbc0010I A

Cbc0010I After 30903000 nodes, 22000 on tree, 191.5 best solution, best possible 113.11938 (70466.34 seconds)
Cbc0010I After 30904000 nodes, 22014 on tree, 191.5 best solution, best possible 113.11938 (70468.18 seconds)
Cbc0010I After 30905000 nodes, 21997 on tree, 191.5 best solution, best possible 113.11938 (70470.43 seconds)
Cbc0010I After 30906000 nodes, 22010 on tree, 191.5 best solution, best possible 113.11938 (70472.43 seconds)
Cbc0010I After 30907000 nodes, 22022 on tree, 191.5 best solution, best possible 113.11938 (70474.49 seconds)
Cbc0010I After 30908000 nodes, 22012 on tree, 191.5 best solution, best possible 113.11938 (70476.51 seconds)
Cbc0010I After 30909000 nodes, 21988 on tree, 191.5 best solution, best possible 113.11938 (70478.57 seconds)
Cbc0010I After 30910000 nodes, 22010 on tree, 191.5 best solution, best possible 113.11938 (70480.52 seconds)
Cbc0010I After 30911000 nodes, 22008 on tree, 191.5 best solution, best possible 113.11938 (70482.44 seconds)
Cbc0010I A

Cbc0010I After 30977000 nodes, 22011 on tree, 191.5 best solution, best possible 113.11938 (70610.38 seconds)
Cbc0010I After 30978000 nodes, 21997 on tree, 191.5 best solution, best possible 113.11938 (70613.22 seconds)
Cbc0010I After 30979000 nodes, 21995 on tree, 191.5 best solution, best possible 113.11938 (70615.75 seconds)
Cbc0010I After 30980000 nodes, 21987 on tree, 191.5 best solution, best possible 113.11938 (70617.87 seconds)
Cbc0010I After 30981000 nodes, 21976 on tree, 191.5 best solution, best possible 113.11938 (70619.79 seconds)
Cbc0010I After 30982000 nodes, 21977 on tree, 191.5 best solution, best possible 113.11938 (70621.75 seconds)
Cbc0010I After 30983000 nodes, 21985 on tree, 191.5 best solution, best possible 113.11938 (70623.76 seconds)
Cbc0010I After 30984000 nodes, 21963 on tree, 191.5 best solution, best possible 113.11938 (70625.71 seconds)
Cbc0010I After 30985000 nodes, 21987 on tree, 191.5 best solution, best possible 113.11938 (70627.88 seconds)
Cbc0010I A

Cbc0010I After 31052000 nodes, 21996 on tree, 191.5 best solution, best possible 113.11938 (70777.14 seconds)
Cbc0010I After 31053000 nodes, 22002 on tree, 191.5 best solution, best possible 113.11938 (70779.34 seconds)
Cbc0010I After 31054000 nodes, 21998 on tree, 191.5 best solution, best possible 113.11938 (70781.50 seconds)
Cbc0010I After 31055000 nodes, 22005 on tree, 191.5 best solution, best possible 113.11938 (70783.72 seconds)
Cbc0010I After 31056000 nodes, 22006 on tree, 191.5 best solution, best possible 113.11938 (70785.90 seconds)
Cbc0010I After 31057000 nodes, 22002 on tree, 191.5 best solution, best possible 113.11938 (70788.18 seconds)
Cbc0010I After 31058000 nodes, 21991 on tree, 191.5 best solution, best possible 113.11938 (70790.35 seconds)
Cbc0010I After 31059000 nodes, 22000 on tree, 191.5 best solution, best possible 113.11938 (70792.69 seconds)
Cbc0010I After 31060000 nodes, 22008 on tree, 191.5 best solution, best possible 113.11938 (70794.96 seconds)
Cbc0010I A

KeyboardInterrupt: 